In [12]:
import os
import pandas as pd

In [13]:
# Define o caminho para a pasta "dados"
folder_path = "dados"

# lista para armazenar os dataframes de cada arquivo CSV
dfs = []

In [14]:
# Percorre todos os arquivos .txt na pasta
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        # lê o arquivo como CSV com o pandas
        df = pd.read_csv(os.path.join(folder_path, filename), delimiter=';')

        # Verificar se o arquivo possui a coluna 'competênciaexc'
        if 'competênciaexc' not in df.columns:
            # Criar a coluna 'competênciaexc' com valor nulo
            df['competênciaexc'] = None

        # Verificar se o arquivo possui a coluna 'indicadordeexclusão'
        if 'indicadordeexclusão' not in df.columns:
            # Criar a coluna 'indicadordeexclusão' com valor 0
            df['indicadordeexclusão'] = 0
        
        # Excluir coluna não usadas
        df = df.drop(['unidadesaláriocódigo', 'valorsaláriofixo'], axis=1)

        # Adiciona o dataframe à lista de dataframes
        dfs.append(df)

In [15]:
# concatena os dataframes em um único dataframe final
df_group = pd.concat(dfs, ignore_index=True)

In [16]:
pd.set_option('display.max_columns', None)

In [17]:
# exibe o dataframe final
df_group.head()

,competênciamov,região,uf,município,seção,subclasse,saldomovimentação,cbo2002ocupação,categoria,graudeinstrução,idade,horascontratuais,raçacor,sexo,tipoempregador,tipoestabelecimento,tipomovimentação,tipodedeficiência,indtrabintermitente,indtrabparcial,salário,tamestabjan,indicadoraprendiz,origemdainformação,competênciadec,competênciaexc,indicadordeexclusão,indicadordeforadoprazo
0,202001,3,35,352930,Q,8630503,1,422105,105,7,27.0,"44,00",1,3,0,1,20,0,0,0,"1200,00",2,0,1,202001,202005,1,0
1,202001,3,33,330630,G,4742300,-1,951105,101,7,29.0,"42,50",1,1,0,1,31,0,0,0,"1836,00",5,0,1,202001,202005,1,0
2,202001,1,15,150140,G,4646001,-1,142305,101,7,39.0,"44,00",3,3,0,1,31,0,0,0,"1998,00",2,0,1,202001,202005,1,0
3,202001,3,33,330455,Q,8660700,-1,515105,105,7,48.0,"48,00",2,3,0,1,43,0,0,0,"1400,00",10,0,1,202001,202005,1,0
4,202001,3,33,330455,Q,8660700,-1,515105,105,8,32.0,"48,00",2,3,0,1,43,0,0,0,"1400,00",10,0,1,202001,202005,1,0


In [8]:
df_group.dtypes

competênciamov              int64
região                      int64
uf                          int64
município                   int64
seção                      object
subclasse                   int64
saldomovimentação           int64
cbo2002ocupação             int64
categoria                   int64
graudeinstrução             int64
idade                     float64
horascontratuais           object
raçacor                     int64
sexo                        int64
tipoempregador              int64
tipoestabelecimento         int64
tipomovimentação            int64
tipodedeficiência           int64
indtrabintermitente         int64
indtrabparcial              int64
salário                    object
tamestabjan                 int64
indicadoraprendiz           int64
origemdainformação          int64
competênciadec              int64
competênciaexc             object
indicadordeexclusão         int64
indicadordeforadoprazo      int64
unidadesaláriocódigo        int64
valorsaláriofi

In [9]:
df_group.columns

Index(['competênciamov', 'região', 'uf', 'município', 'seção', 'subclasse',
       'saldomovimentação', 'cbo2002ocupação', 'categoria', 'graudeinstrução',
       'idade', 'horascontratuais', 'raçacor', 'sexo', 'tipoempregador',
       'tipoestabelecimento', 'tipomovimentação', 'tipodedeficiência',
       'indtrabintermitente', 'indtrabparcial', 'salário', 'tamestabjan',
       'indicadoraprendiz', 'origemdainformação', 'competênciadec',
       'competênciaexc', 'indicadordeexclusão', 'indicadordeforadoprazo',
       'unidadesaláriocódigo', 'valorsaláriofixo'],
      dtype='object')

In [10]:
df_group.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1948000 entries, 0 to 1947999
Data columns (total 30 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   competênciamov          int64  
 1   região                  int64  
 2   uf                      int64  
 3   município               int64  
 4   seção                   object 
 5   subclasse               int64  
 6   saldomovimentação       int64  
 7   cbo2002ocupação         int64  
 8   categoria               int64  
 9   graudeinstrução         int64  
 10  idade                   float64
 11  horascontratuais        object 
 12  raçacor                 int64  
 13  sexo                    int64  
 14  tipoempregador          int64  
 15  tipoestabelecimento     int64  
 16  tipomovimentação        int64  
 17  tipodedeficiência       int64  
 18  indtrabintermitente     int64  
 19  indtrabparcial          int64  
 20  salário                 object 
 21  tamestabjan             int64  

In [11]:
df_group.shape

(1948000, 30)

In [12]:
df_group.isna().sum()

competênciamov                  0
região                          0
uf                              0
município                       0
seção                           0
subclasse                       0
saldomovimentação               0
cbo2002ocupação                 0
categoria                       0
graudeinstrução                 0
idade                           5
horascontratuais             5087
raçacor                         0
sexo                            0
tipoempregador                  0
tipoestabelecimento             0
tipomovimentação                0
tipodedeficiência               0
indtrabintermitente             0
indtrabparcial                  0
salário                         0
tamestabjan                     0
indicadoraprendiz               0
origemdainformação              0
competênciadec                  0
competênciaexc            1944129
indicadordeexclusão             0
indicadordeforadoprazo          0
unidadesaláriocódigo            0
valorsaláriofi